In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from nets import alexnet

In [2]:
def read_and_decode(file, batch_size, one_hot=False):
    file_queue = tf.train.string_input_producer([file])
    reader = tf.TFRecordReader()
    
    _, serialized = reader.read(file_queue)
    features = tf.parse_single_example(serialized, features={
            "image": tf.FixedLenFeature([], tf.string),
            "label": tf.FixedLenFeature([], tf.int64)
        })
    image = tf.decode_raw(features["image"], tf.uint8)
    image = tf.reshape(image, [224, 224, 3])
    label = tf.cast(features["label"], tf.int32)
    
    min_after_dequeue = 2 * batch_size
    capacity = 3 * batch_size + min_after_dequeue
    
    images, labels = tf.train.shuffle_batch([image, label], batch_size=batch_size, min_after_dequeue=min_after_dequeue, capacity=capacity)
    
    if one_hot:
        labels = tf.one_hot(labels, 2)
    
    return images, labels


class AlexNet(object):
    def __init__(self, dataset_file, num_classes, learning_rate=1e-4, train_steps=200, batch_size=50, keep_prob=0.5, images_dir=None):
        self.dataset_file = dataset_file
        self.lr = learning_rate
        self.keep_prob = keep_prob
        self.train_steps = train_steps
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.weights = {
            "conv1": tf.Variable(tf.truncated_normal([11, 11, 3, 96], stddev=0.1)),
            "conv2": tf.Variable(tf.truncated_normal([5, 5, 96, 256], stddev=0.1)),
            "conv3": tf.Variable(tf.truncated_normal([3, 3, 256, 384], stddev=0.1)),
            "conv4": tf.Variable(tf.truncated_normal([3, 3, 384, 384], stddev=0.1)),
            "conv5": tf.Variable(tf.truncated_normal([3, 3, 384, 256], stddev=0.1)),
            "fc6": tf.Variable(tf.truncated_normal([6 * 6 * 256, 4096], stddev=0.1)),
            "fc7": tf.Variable(tf.truncated_normal([4096, 2048], stddev=0.1)),
            "fc8": tf.Variable(tf.truncated_normal([2048, self.num_classes], stddev=0.1))
        }
        self.biases = {
            "conv1": tf.Variable(tf.constant(0.0, shape=[96])),
            "conv2": tf.Variable(tf.constant(0.1, shape=[256])),
            "conv3": tf.Variable(tf.constant(0.1, shape=[384])),
            "conv4": tf.Variable(tf.constant(0.1, shape=[384])),
            "conv5": tf.Variable(tf.constant(0.1, shape=[256])),
            "fc6": tf.Variable(tf.constant(0.1, shape=[4096])),
            "fc7": tf.Variable(tf.constant(0.1, shape=[2048])),
            "fc8": tf.Variable(tf.constant(0.0, shape=[self.num_classes]))
        }
    
#     def resize_images(src_dir, dest_dir):
#         """
#         resize images to 227x227 shape
#         """
#         if not os.path.exists(src_dir):
#             sys.stderr.write("Dir %s doesn\'t existing.\n" % src_dir)
#             return
        
#         if not os.path.exists(dest_dir):
#             os.makedirs(dest_dir)
        
#         for root, subdirs, files in os.walk(src_dir):
#             for filename in files:
#                 filepath = os.path.join(root, filename)
#                 output_filepath = os.path.join(dest_dir, filename)
#                 try:
#                     image = cv2.imread(filepath)
#                     dim = (227, 227)
#                     image_resized = cv2.resize(image, dim)
#                     cv2.imwrite(output_filepath, image_resized)
#                 except:
#                     print("Fail to resize %s, skip it." % filepath)
#             cv2.waitKey(0)

    def predict(self, X):
        X_image = tf.reshape(X, [-1, 227, 227, 3])
        
        # conv1 layer
        with tf.name_scope("conv1"):
            conv = tf.nn.conv2d(X_image, self.weights["conv1"], [1, 4, 4, 1], "VALID")
            net = tf.nn.bias_add(conv, self.biases["conv1"])
            output = tf.nn.relu(net)
            pool = tf.nn.max_pool(output, [1, 3, 3, 1], [1, 2, 2, 1], 'VALID')
            conv1_y = tf.nn.lrn(pool, alpha=0.0001, beta=0.75)
        
        # conv2 layer
        with tf.name_scope("conv2"):
            conv = tf.nn.conv2d(conv1_y, self.weights["conv2"], [1, 1, 1, 1], "SAME")
            net = tf.nn.bias_add(conv, self.biases["conv2"])
            output = tf.nn.relu(net)
            pool = tf.nn.max_pool(output, [1, 3, 3, 1], [1, 2, 2, 1], 'VALID')
            conv2_y = tf.nn.lrn(pool, alpha=0.0001, beta=0.75)
            
        # conv3 layer
        with tf.name_scope("conv3"):
            conv = tf.nn.conv2d(conv2_y, self.weights["conv3"], [1, 1, 1, 1], "SAME")
            net = tf.nn.bias_add(conv, self.biases["conv3"])
            conv3_y = tf.nn.relu(net)
                
        # conv4 layer
        with tf.name_scope("conv4"):
            conv = tf.nn.conv2d(conv3_y, self.weights["conv4"], [1, 1, 1, 1], "SAME")
            net = tf.nn.bias_add(conv, self.biases["conv4"])
            conv4_y = tf.nn.relu(net)
        
        # conv5 layer
        with tf.name_scope("conv5"):
            conv = tf.nn.conv2d(conv4_y, self.weights["conv5"], [1, 1, 1, 1], "SAME")
            net = tf.nn.bias_add(conv, self.biases["conv5"])
            output = tf.nn.relu(net)
            conv5_y = tf.nn.max_pool(output, [1, 3, 3, 1], [1, 2, 2, 1], 'VALID')
        
        # fc6 layer
        with tf.name_scope("fc6"):
            v = conv5_y.get_shape().as_list()
            conv5_y = tf.reshape(conv5_y, [-1, v[1] * v[2] * v[3]])
            net = tf.matmul(conv5_y, self.weights["fc6"]) + self.biases["fc6"]
            fc6_y = tf.nn.relu(net)
            fc6_y = tf.nn.dropout(fc6_y, self.keep_prob)
        
        # fc7 layer
        with tf.name_scope("fc7"):
            net = tf.matmul(fc6_y, self.weights["fc7"]) + self.biases["fc7"]
            fc7_y = tf.nn.relu(net)
            fc7_y = tf.nn.dropout(fc7_y, self.keep_prob)
            
        # fc8 layer
        with tf.name_scope("fc8"):
            net = tf.matmul(fc7_y, self.weights["fc8"]) + self.biases["fc8"]
            fc8_y = tf.nn.softmax(net)
            
        return fc8_y

In [ ]:
TRAIN_DS_FILE = "data/train.tfrecord"
BATCH_SIZE = 50
EPOCH = 6000

X = tf.placeholder(tf.float32, [None, 224, 224, 3])
y = tf.placeholder(tf.int32, [None, 2])
lr = tf.Variable(0.002, tf.float32)

# nn = AlexNet(TRAIN_DS_FILE, num_classes=2)
# image_batch, label_batch = read_and_decode(nn.dataset_file, nn.batch_size, one_hot=True)
# y_hat = nn.predict(X)

image_batch, label_batch = read_and_decode(TRAIN_DS_FILE, BATCH_SIZE, one_hot=True)
with slim.arg_scope(alexnet.alexnet_v2_arg_scope()):
    y_hat, _ = alexnet.alexnet_v2(X, 2)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_hat))
train_op = tf.train.AdamOptimizer(lr).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    sess.run(tf.global_variables_initializer())
    
    for i in range(1, EPOCH + 1):
        images, labels = sess.run([image_batch, label_batch])
        sess.run(train_op, feed_dict={X: images, y: labels})
        if i % 20 == 0:
            if i % 1000 == 0:
                sess.run(tf.assign(lr, lr / 3))
            cost, acc = sess.run([loss, accuracy], feed_dict={X: images, y: labels})
            print("epoch: %d, cost: %.5f, accuracy: %.3f" % (i, cost, acc * 100))
    
    coord.request_stop()
    coord.join(threads)